- 다중 control signal을 사용하는 model을 만들기 위한 곳
- 현재 분석으로는 256 by 256으로 finetune 되었다는 점이 크게 작용한다.
- 약간의 variant라고 한다면, "model.diffusion_model.input_blocks.0.0.weight" 이 부분이 input dimension이 8이 되었기 떄문에 avearage를 취해야 겠다.
- 학습 scale이 lab scale은 아니고 중소기업 정도는 되기 때문에, 역시 이를 control signal형태로 사용하는 것은 적절한 관측이 될 것으로 보인다.
- 그래도 30K scale의 finetune을 거치기 때문에, spatial resolution에 의한 문제는 크게 발생하지 않을 것으로 보인다.
- [ ] control signal composition에 대한 실험만 거치면 될 것으로 보인다

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import json
import pytorch_lightning as pl
import torch
from omegaconf import OmegaConf

import copy

repo_path = os.path.abspath(os.path.join('..'))
sys.path.append(repo_path)

repo_path_2 = "/workspace/code/3DAnything/zero123"
sys.path.append(repo_path_2)

from zero123.ldm.util import instantiate_from_config

/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
def convert_state_dict_to_jsonl(state_dict, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as f:
        for key, value in state_dict.items():
            json_line = {"key": key, "shape": value.shape}
            json.dump(json_line, f, ensure_ascii=False)
            f.write('\n')

In [6]:
test_cfg_path = "/workspace/code/3DAnything/zero123/configs/canny-edge.yaml"
test_cfg = OmegaConf.load(test_cfg_path)

In [6]:
test_orig_cfg_path = "/workspace/code/3DAnything/zero123/configs/sd-objaverse-finetune-c_concat-256.yaml"
test_orig_cfg = OmegaConf.load(test_orig_cfg_path)

In [ ]:
model = instantiate_from_config(test_cfg.model)

ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


/opt/conda/lib/python3.10/site-packages/clip/clip.py:57: UserWarning: /root/.cache/clip/ViT-L-14.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(f"{download_target} exists, but the SHA256 checksum does not match; re-downloading the file")
100%|████████████████████████████████████████| 890M/890M [20:48<00:00, 747kiB/s]


In [7]:
model_orig = instantiate_from_config(test_orig_cfg.model)

LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.53 M params.
Keeping EMAs of 688.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


NameError: name 'diffusion_model' is not defined

In [8]:
model_orig

LatentDiffusion(
  (model): DiffusionWrapper(
    (diffusion_model): UNetModel(
      (time_embed): Sequential(
        (0): Linear(in_features=320, out_features=1280, bias=True)
        (1): SiLU()
        (2): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (input_blocks): ModuleList(
        (0): TimestepEmbedSequential(
          (0): Conv2d(8, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (1-2): 2 x TimestepEmbedSequential(
          (0): ResBlock(
            (in_layers): Sequential(
              (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
              (1): SiLU()
              (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            )
            (h_upd): Identity()
            (x_upd): Identity()
            (emb_layers): Sequential(
              (0): SiLU()
              (1): Linear(in_features=1280, out_features=320, bias=True)
            )
            (out_layers): Sequential(
            

In [35]:
convert_state_dict_to_jsonl(model_orig.state_dict(), "model_orig_state_dict.jsonl")

In [3]:
model_orig_weight = torch.load("/workspace/weight/diffuse/zero123-weights/165000.ckpt", map_location="cpu")["state_dict"]

In [ ]:
model_orig_weight

In [ ]:
# IMPORTANT
new_zero123_checkpoint = {} 
for k, v in model_orgi_stdict.items():
    if ("diffusion_model" in k and "model_ema" not in k) and ("input_blocks" in k or "middle_block" in k):
        _k = k.replace("model.diffusion_model.", "")
        
        if "input_blocks.0.0.weight" in _k:
            # (320, 8, 3, 3) -> (320, 4, 2, 3, 3) -> (320, 4, 3, 3)
            v_shape = v.shape
            v = v.view(v_shape[0], v_shape[1] /2, 2, v_shape[2], v_shape[3])
            v = v.mean(dim=2)
        
        if "zero_convs" in _k:
            v = torch.zeros_like(v)
            
        new_zero123_checkpoint[_k] = v

In [ ]:
new_zero123_checkpoint.keys()

dict_keys(['input_blocks.0.0.weight', 'input_blocks.0.0.bias', 'input_blocks.1.0.in_layers.0.weight', 'input_blocks.1.0.in_layers.0.bias', 'input_blocks.1.0.in_layers.2.weight', 'input_blocks.1.0.in_layers.2.bias', 'input_blocks.1.0.emb_layers.1.weight', 'input_blocks.1.0.emb_layers.1.bias', 'input_blocks.1.0.out_layers.0.weight', 'input_blocks.1.0.out_layers.0.bias', 'input_blocks.1.0.out_layers.3.weight', 'input_blocks.1.0.out_layers.3.bias', 'input_blocks.1.1.norm.weight', 'input_blocks.1.1.norm.bias', 'input_blocks.1.1.proj_in.weight', 'input_blocks.1.1.proj_in.bias', 'input_blocks.1.1.transformer_blocks.0.attn1.to_q.weight', 'input_blocks.1.1.transformer_blocks.0.attn1.to_k.weight', 'input_blocks.1.1.transformer_blocks.0.attn1.to_v.weight', 'input_blocks.1.1.transformer_blocks.0.attn1.to_out.0.weight', 'input_blocks.1.1.transformer_blocks.0.attn1.to_out.0.bias', 'input_blocks.1.1.transformer_blocks.0.ff.net.0.proj.weight', 'input_blocks.1.1.transformer_blocks.0.ff.net.0.proj.bias'

In [ ]:
test_weight = copy.copy(new_zero123_checkpoint[])

In [81]:
test_weight.reshape(320, 4, 2, 3, 3).mean(dim=2).shape

torch.Size([320, 4, 3, 3])

In [75]:
test_tensor = torch.randn(1, 3, 256, 256)
test_tensor = test_tensor.to("cuda")

In [ ]:
# model.first_stage_model
vae = model.first_stage_model.to("cuda")
# 여기는 그대로 가져오기

In [81]:
vae_2 = model_orig.first_stage_model.to("cuda")

In [82]:
result = vae.encode(test_tensor)
result_2 = vae_2.encode(test_tensor)

In [88]:
result_sampled = result.sample()

In [90]:
result_sampled.shape

torch.Size([1, 4, 32, 32])

In [84]:
# result.mean.shape
result_2.mean.shape

torch.Size([1, 4, 32, 32])

In [ ]:
model.model.diffusion_model
# 여기를 T2I 모델로 바꿔야 함

ControlledUnetModel(
  (time_embed): Sequential(
    (0): Linear(in_features=320, out_features=1280, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (input_blocks): ModuleList(
    (0): TimestepEmbedSequential(
      (0): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1-2): 2 x TimestepEmbedSequential(
      (0): ResBlock(
        (in_layers): Sequential(
          (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (h_upd): Identity()
        (x_upd): Identity()
        (emb_layers): Sequential(
          (0): SiLU()
          (1): Linear(in_features=1280, out_features=320, bias=True)
        )
        (out_layers): Sequential(
          (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Dropout(p=0, inplace=False)
          (3): Conv2d(320, 320, kernel

In [ ]:
model.model.diffusion_model.input_blocks

ModuleList(
  (0): TimestepEmbedSequential(
    (0): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (1-2): 2 x TimestepEmbedSequential(
    (0): ResBlock(
      (in_layers): Sequential(
        (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
        (1): SiLU()
        (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (h_upd): Identity()
      (x_upd): Identity()
      (emb_layers): Sequential(
        (0): SiLU()
        (1): Linear(in_features=1280, out_features=320, bias=True)
      )
      (out_layers): Sequential(
        (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
        (1): SiLU()
        (2): Dropout(p=0, inplace=False)
        (3): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (skip_connection): Identity()
    )
    (1): SpatialTransformer(
      (norm): GroupNorm(32, 320, eps=1e-06, affine=True)
      (proj_in): Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1))
      

In [ ]:
model.cond_stage_model
# 이 부분의 clip을 치환하면 된다.

FrozenCLIPImageEmbedder(
  (model): CLIP(
    (visual): VisionTransformer(
      (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): Sequential(
          (0): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
            )
            (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
              (gelu): QuickGELU()
              (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
            )
            (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          )
          (1): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonD

In [113]:
model.control_model.zero_convs
# 여기는 code implementation을 약간 바꿔야 한다. 이종 control을 둘 다 넣기 위해서

ModuleList(
  (0-3): 4 x TimestepEmbedSequential(
    (0): Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1))
  )
  (4-6): 3 x TimestepEmbedSequential(
    (0): Conv2d(640, 640, kernel_size=(1, 1), stride=(1, 1))
  )
  (7-11): 5 x TimestepEmbedSequential(
    (0): Conv2d(1280, 1280, kernel_size=(1, 1), stride=(1, 1))
  )
)

# test_orig

In [74]:
model_orig.first_stage_model


AutoencoderKL(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down): ModuleList(
      (0): Module(
        (block): ModuleList(
          (0-1): 2 x ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
        )
        (attn): ModuleList()
        (downsample): Downsample(
          (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
        )
      )
      (1): Module(
        (block): ModuleList(
          (0): ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (no

In [72]:
model_orig.model.diffusion_model

UNetModel(
  (time_embed): Sequential(
    (0): Linear(in_features=320, out_features=1280, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (input_blocks): ModuleList(
    (0): TimestepEmbedSequential(
      (0): Conv2d(8, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1-2): 2 x TimestepEmbedSequential(
      (0): ResBlock(
        (in_layers): Sequential(
          (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (h_upd): Identity()
        (x_upd): Identity()
        (emb_layers): Sequential(
          (0): SiLU()
          (1): Linear(in_features=1280, out_features=320, bias=True)
        )
        (out_layers): Sequential(
          (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Dropout(p=0, inplace=False)
          (3): Conv2d(320, 320, kernel_size=(3, 

In [87]:
model_orig.model.diffusion_model.input_blocks[0][0].weight.shape
# 여기 convolution layer만 control하면 될 듯

torch.Size([320, 8, 3, 3])

# SD weight 분석

In [11]:
sd_path = "/workspace/weight/diffuse/stable-diffusion-v-1-4-original/sd-v1-4.ckpt"
sd_cfg_path = "/workspace/code/3DAnything/zero123/configs/sd-v1-inference.yaml"
sd_cfg = OmegaConf.load(sd_cfg_path)


In [96]:
sd_cfg

{'model': {'base_learning_rate': 0.0001, 'target': 'ldm.models.diffusion.ddpm.LatentDiffusion', 'params': {'linear_start': 0.00085, 'linear_end': 0.012, 'num_timesteps_cond': 1, 'log_every_t': 200, 'timesteps': 1000, 'first_stage_key': 'jpg', 'cond_stage_key': 'txt', 'image_size': 64, 'channels': 4, 'cond_stage_trainable': False, 'conditioning_key': 'crossattn', 'monitor': 'val/loss_simple_ema', 'scale_factor': 0.18215, 'use_ema': False, 'scheduler_config': {'target': 'ldm.lr_scheduler.LambdaLinearScheduler', 'params': {'warm_up_steps': [10000], 'cycle_lengths': [10000000000000], 'f_start': [1e-06], 'f_max': [1.0], 'f_min': [1.0]}}, 'unet_config': {'target': 'ldm.modules.diffusionmodules.openaimodel.UNetModel', 'params': {'image_size': 32, 'in_channels': 4, 'out_channels': 4, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_heads': 8, 'use_spatial_transformer': True, 'transformer_depth': 1, 'context_dim': 768, 'use_check

In [12]:
sd_model = instantiate_from_config(sd_cfg.model)

LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


In [13]:
test_weight = torch.load(sd_path, map_location="cpu")["state_dict"]

In [ ]:
print(test_weight.keys())
convert_state_dict_to_jsonl(test_weight, "sd_v1_4_weights.jsonl")

dict_keys(['betas', 'alphas_cumprod', 'alphas_cumprod_prev', 'sqrt_alphas_cumprod', 'sqrt_one_minus_alphas_cumprod', 'log_one_minus_alphas_cumprod', 'sqrt_recip_alphas_cumprod', 'sqrt_recipm1_alphas_cumprod', 'posterior_variance', 'posterior_log_variance_clipped', 'posterior_mean_coef1', 'posterior_mean_coef2', 'model.diffusion_model.time_embed.0.weight', 'model.diffusion_model.time_embed.0.bias', 'model.diffusion_model.time_embed.2.weight', 'model.diffusion_model.time_embed.2.bias', 'model.diffusion_model.input_blocks.0.0.weight', 'model.diffusion_model.input_blocks.0.0.bias', 'model.diffusion_model.input_blocks.1.0.in_layers.0.weight', 'model.diffusion_model.input_blocks.1.0.in_layers.0.bias', 'model.diffusion_model.input_blocks.1.0.in_layers.2.weight', 'model.diffusion_model.input_blocks.1.0.in_layers.2.bias', 'model.diffusion_model.input_blocks.1.0.emb_layers.1.weight', 'model.diffusion_model.input_blocks.1.0.emb_layers.1.bias', 'model.diffusion_model.input_blocks.1.0.out_layers.0.

In [99]:
sd_model.model.diffusion_model.input_blocks[0][0].weight.shape

torch.Size([320, 4, 3, 3])

In [63]:
sd_model.load_state_dict(test_weight, strict=False)
# 'cond_stage_model.transformer.text_model.embeddings.position_ids' 이것 말고는 적절하게 load된다.

NameError: name 'sd_model' is not defined

In [14]:
sd_model

LatentDiffusion(
  (model): DiffusionWrapper(
    (diffusion_model): UNetModel(
      (time_embed): Sequential(
        (0): Linear(in_features=320, out_features=1280, bias=True)
        (1): SiLU()
        (2): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (input_blocks): ModuleList(
        (0): TimestepEmbedSequential(
          (0): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (1-2): 2 x TimestepEmbedSequential(
          (0): ResBlock(
            (in_layers): Sequential(
              (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
              (1): SiLU()
              (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            )
            (h_upd): Identity()
            (x_upd): Identity()
            (emb_layers): Sequential(
              (0): SiLU()
              (1): Linear(in_features=1280, out_features=320, bias=True)
            )
            (out_layers): Sequential(
            

# ControlNet wieight 분석

In [ ]:
control_canny_path = os.path.join("/workspace/weight/diffuse/ControlNet/models" ,"control_sd15_canny.pth") 

In [20]:
control_net_ckpt = torch.load(control_canny_path, map_location="cpu")

In [110]:
print(control_net_ckpt.keys())
convert_state_dict_to_jsonl(control_net_ckpt, "control_sd15_canny_weights.jsonl")
# json으로 따서 보기

odict_keys(['betas', 'alphas_cumprod', 'alphas_cumprod_prev', 'sqrt_alphas_cumprod', 'sqrt_one_minus_alphas_cumprod', 'log_one_minus_alphas_cumprod', 'sqrt_recip_alphas_cumprod', 'sqrt_recipm1_alphas_cumprod', 'posterior_variance', 'posterior_log_variance_clipped', 'posterior_mean_coef1', 'posterior_mean_coef2', 'logvar', 'model.diffusion_model.time_embed.0.weight', 'model.diffusion_model.time_embed.0.bias', 'model.diffusion_model.time_embed.2.weight', 'model.diffusion_model.time_embed.2.bias', 'model.diffusion_model.input_blocks.0.0.weight', 'model.diffusion_model.input_blocks.0.0.bias', 'model.diffusion_model.input_blocks.1.0.in_layers.0.weight', 'model.diffusion_model.input_blocks.1.0.in_layers.0.bias', 'model.diffusion_model.input_blocks.1.0.in_layers.2.weight', 'model.diffusion_model.input_blocks.1.0.in_layers.2.bias', 'model.diffusion_model.input_blocks.1.0.emb_layers.1.weight', 'model.diffusion_model.input_blocks.1.0.emb_layers.1.bias', 'model.diffusion_model.input_blocks.1.0.ou

In [ ]:
# IMPORTANT
new_canny_checkpoint = {} 
for k, v in control_net_ckpt.items():
    if "control_model" in k:
        _k = k.replace("control_model.", "")
        if "zero_convs" in _k:
            v = torch.zeros_like(v)
        
        new_canny_checkpoint[_k] = v
        

In [32]:
print(new_canny_checkpoint.keys())

dict_keys(['time_embed.0.weight', 'time_embed.0.bias', 'time_embed.2.weight', 'time_embed.2.bias', 'input_blocks.0.0.weight', 'input_blocks.0.0.bias', 'input_blocks.1.0.in_layers.0.weight', 'input_blocks.1.0.in_layers.0.bias', 'input_blocks.1.0.in_layers.2.weight', 'input_blocks.1.0.in_layers.2.bias', 'input_blocks.1.0.emb_layers.1.weight', 'input_blocks.1.0.emb_layers.1.bias', 'input_blocks.1.0.out_layers.0.weight', 'input_blocks.1.0.out_layers.0.bias', 'input_blocks.1.0.out_layers.3.weight', 'input_blocks.1.0.out_layers.3.bias', 'input_blocks.1.1.norm.weight', 'input_blocks.1.1.norm.bias', 'input_blocks.1.1.proj_in.weight', 'input_blocks.1.1.proj_in.bias', 'input_blocks.1.1.transformer_blocks.0.attn1.to_q.weight', 'input_blocks.1.1.transformer_blocks.0.attn1.to_k.weight', 'input_blocks.1.1.transformer_blocks.0.attn1.to_v.weight', 'input_blocks.1.1.transformer_blocks.0.attn1.to_out.0.weight', 'input_blocks.1.1.transformer_blocks.0.attn1.to_out.0.bias', 'input_blocks.1.1.transformer_bl

# 이종 ControlNet Signaling try

In [92]:
control_scratch_path = os.path.join(
    "/workspace/code/3DAnything/zero123/configs/canny-edge_variant_i2t_multi.yaml"
)
control_scratch_cfg= OmegaConf.load(control_scratch_path)

In [94]:
type(control_scratch_cfg.pre_control)

omegaconf.dictconfig.DictConfig

In [ ]:
model_scratch = instantiate_from_config(control_scratch_cfg.model)

MultiControlNet: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


In [29]:
model_scratch.control_model_list[1]

ControlNet(
  (time_embed): Sequential(
    (0): Linear(in_features=320, out_features=1280, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (input_blocks): ModuleList(
    (0): TimestepEmbedSequential(
      (0): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1-2): 2 x TimestepEmbedSequential(
      (0): ResBlock(
        (in_layers): Sequential(
          (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (h_upd): Identity()
        (x_upd): Identity()
        (emb_layers): Sequential(
          (0): SiLU()
          (1): Linear(in_features=1280, out_features=320, bias=True)
        )
        (out_layers): Sequential(
          (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Dropout(p=0, inplace=False)
          (3): Conv2d(320, 320, kernel_size=(3,

In [ ]:
print([k for k, v in model_scratch.control_model_list[0].named_parameters()])

'input_blocks.0.0.weight'

['time_embed.0.weight', 'time_embed.0.bias', 'time_embed.2.weight', 'time_embed.2.bias', 'input_blocks.0.0.weight', 'input_blocks.0.0.bias', 'input_blocks.1.0.in_layers.0.weight', 'input_blocks.1.0.in_layers.0.bias', 'input_blocks.1.0.in_layers.2.weight', 'input_blocks.1.0.in_layers.2.bias', 'input_blocks.1.0.emb_layers.1.weight', 'input_blocks.1.0.emb_layers.1.bias', 'input_blocks.1.0.out_layers.0.weight', 'input_blocks.1.0.out_layers.0.bias', 'input_blocks.1.0.out_layers.3.weight', 'input_blocks.1.0.out_layers.3.bias', 'input_blocks.1.1.norm.weight', 'input_blocks.1.1.norm.bias', 'input_blocks.1.1.proj_in.weight', 'input_blocks.1.1.proj_in.bias', 'input_blocks.1.1.transformer_blocks.0.attn1.to_q.weight', 'input_blocks.1.1.transformer_blocks.0.attn1.to_k.weight', 'input_blocks.1.1.transformer_blocks.0.attn1.to_v.weight', 'input_blocks.1.1.transformer_blocks.0.attn1.to_out.0.weight', 'input_blocks.1.1.transformer_blocks.0.attn1.to_out.0.bias', 'input_blocks.1.1.transformer_blocks.0.ff.

In [66]:
result = model_scratch.load_state_dict(test_weight, strict=False)

In [72]:
result2 = model_scratch.control_model_list[1].load_state_dict(new_canny_checkpoint, strict=False)

In [ ]:
test_weight["input_blocks.0.0.weight"].shape

torch.Size([320, 4, 3, 3])

In [69]:
for key in result.missing_keys:
    print(key)
print("===")
for key in result.unexpected_keys:
    print(key)

cc_projection.weight
cc_projection.bias
control_model_list.0.time_embed.0.weight
control_model_list.0.time_embed.0.bias
control_model_list.0.time_embed.2.weight
control_model_list.0.time_embed.2.bias
control_model_list.0.input_blocks.0.0.weight
control_model_list.0.input_blocks.0.0.bias
control_model_list.0.input_blocks.1.0.in_layers.0.weight
control_model_list.0.input_blocks.1.0.in_layers.0.bias
control_model_list.0.input_blocks.1.0.in_layers.2.weight
control_model_list.0.input_blocks.1.0.in_layers.2.bias
control_model_list.0.input_blocks.1.0.emb_layers.1.weight
control_model_list.0.input_blocks.1.0.emb_layers.1.bias
control_model_list.0.input_blocks.1.0.out_layers.0.weight
control_model_list.0.input_blocks.1.0.out_layers.0.bias
control_model_list.0.input_blocks.1.0.out_layers.3.weight
control_model_list.0.input_blocks.1.0.out_layers.3.bias
control_model_list.0.input_blocks.1.1.norm.weight
control_model_list.0.input_blocks.1.1.norm.bias
control_model_list.0.input_blocks.1.1.proj_in.we

In [86]:
model_scratch.control_model_list[1].zero_convs[0][0].weight

Parameter containing:
tensor([[[[-3.2378e-04]],

         [[ 5.4660e-03]],

         [[-5.6703e-03]],

         ...,

         [[ 1.9892e-03]],

         [[ 4.2112e-03]],

         [[-5.7546e-03]]],


        [[[ 1.9025e-03]],

         [[-6.9791e-03]],

         [[ 4.2329e-03]],

         ...,

         [[ 2.1953e-03]],

         [[ 5.1083e-03]],

         [[ 6.1575e-03]]],


        [[[ 3.2860e-05]],

         [[ 8.7645e-04]],

         [[ 2.5143e-04]],

         ...,

         [[-5.7085e-03]],

         [[-4.1409e-04]],

         [[-4.8830e-03]]],


        ...,


        [[[-4.1807e-03]],

         [[-3.6767e-03]],

         [[ 6.2936e-03]],

         ...,

         [[ 1.0634e-03]],

         [[-5.2150e-03]],

         [[ 4.4617e-03]]],


        [[[ 1.2845e-03]],

         [[ 6.9236e-03]],

         [[-5.4860e-03]],

         ...,

         [[-5.9793e-03]],

         [[ 1.2769e-02]],

         [[-7.1989e-03]]],


        [[[-1.2720e-03]],

         [[-8.9656e-03]],

         [[-2.

In [73]:
for key in result2.missing_keys:
    print(key)
print("===")
for key in result2.unexpected_keys:
    print(key)

===


# load function 짜기
- 처음에는 stable diffusion origin
- 이후에는 각각의 controlNet model들에 대하여
    - 다만, zero123는 8 -> 4 시에 2개 channel씩 average 해주기 

In [ ]:
model_scratch = instantiate_from_config(control_scratch_cfg.model)

MultiControlNet: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


ConfigAttributeError: Missing key itmes
    full_key: pre_control.path.itmes
    object_type=dict

In [112]:
model_scratch.load_pre_control(pre_control_cfg=control_scratch_cfg.pre_control, debug=True)

Loading sd control model weights
sd Control model weights: _IncompatibleKeys(missing_keys=['cc_projection.weight', 'cc_projection.bias', 'control_model_list.0.time_embed.0.weight', 'control_model_list.0.time_embed.0.bias', 'control_model_list.0.time_embed.2.weight', 'control_model_list.0.time_embed.2.bias', 'control_model_list.0.input_blocks.0.0.weight', 'control_model_list.0.input_blocks.0.0.bias', 'control_model_list.0.input_blocks.1.0.in_layers.0.weight', 'control_model_list.0.input_blocks.1.0.in_layers.0.bias', 'control_model_list.0.input_blocks.1.0.in_layers.2.weight', 'control_model_list.0.input_blocks.1.0.in_layers.2.bias', 'control_model_list.0.input_blocks.1.0.emb_layers.1.weight', 'control_model_list.0.input_blocks.1.0.emb_layers.1.bias', 'control_model_list.0.input_blocks.1.0.out_layers.0.weight', 'control_model_list.0.input_blocks.1.0.out_layers.0.bias', 'control_model_list.0.input_blocks.1.0.out_layers.3.weight', 'control_model_list.0.input_blocks.1.0.out_layers.3.bias', '

In [113]:
import torch.nn as nn

test = nn.ModuleList()
test.append(
   nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False) 
)
test.append(
   nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False) 
)

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)

In [114]:
test.parameters()

<generator object Module.parameters at 0x7ff04c5c3ed0>

In [16]:
test_path = "/workspace/code/3DAnything/zero123/configs/canny-edge_variant_i2t_multimodal_v3_512.yaml"
test_cfg = OmegaConf.load(test_path)

In [17]:
new_model = instantiate_from_config(test_cfg.model)

MultiModalControlNetV3: Running in eps-prediction mode
DiffusionWrapper has 859.53 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 64, 64) = 16384 dimensions.
making attention of type 'vanilla' with 512 in_channels


In [18]:
new_model.load_pre_control(pre_control_cfg=test_cfg.pre_control, debug=True)

Loading sd control model weights
sd Control model weights: _IncompatibleKeys(missing_keys=['zero_convs.0.0.weight', 'zero_convs.0.0.bias', 'zero_convs.1.0.weight', 'zero_convs.1.0.bias', 'zero_convs.2.0.weight', 'zero_convs.2.0.bias', 'zero_convs.3.0.weight', 'zero_convs.3.0.bias', 'zero_convs.4.0.weight', 'zero_convs.4.0.bias', 'zero_convs.5.0.weight', 'zero_convs.5.0.bias', 'zero_convs.6.0.weight', 'zero_convs.6.0.bias', 'zero_convs.7.0.weight', 'zero_convs.7.0.bias', 'zero_convs.8.0.weight', 'zero_convs.8.0.bias', 'zero_convs.9.0.weight', 'zero_convs.9.0.bias', 'zero_convs.10.0.weight', 'zero_convs.10.0.bias', 'zero_convs.11.0.weight', 'zero_convs.11.0.bias', 'middle_block_out.0.weight', 'middle_block_out.0.bias'], unexpected_keys=[])
Loading pose control model weights
pose Control model weights: _IncompatibleKeys(missing_keys=['cond_stage_model.txt.transformer.text_model.embeddings.token_embedding.weight', 'cond_stage_model.txt.transformer.text_model.embeddings.position_embedding.w